In [1]:
from src.RawDataLoaders import CAsT_RawDataLoader
from src.models_and_transforms.text_transforms import Rewriter_Query_Resolver_Transform, Rewriter_Context_Query_Merge_Transform, \
                                                      BART_Numericalise_Transform
from src.pipe_datasets import CAsT_Query_ReWriting_Dataset
from src.models_and_transforms.BART_models import BART_Query_ReWriter
from src.trainers import Model_Trainer

from transformers import BertTokenizer, BartTokenizer
from pytorch_lightning import Trainer, Callback
from pytorch_lightning.loggers import WandbLogger
from tqdm.auto import tqdm 
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
raw_data_loader = CAsT_RawDataLoader()
get_query_fn = raw_data_loader.get_query
get_doc_fn = raw_data_loader.get_doc

In [3]:
train_raw_samples = raw_data_loader.get_topics("train")

In [4]:
train_samples = Rewriter_Query_Resolver_Transform(get_query_fn)(train_raw_samples)
train_samples = Rewriter_Context_Query_Merge_Transform()(train_samples)
train_samples = BART_Numericalise_Transform(fields=[('input_text', 'input_ids'), ('resolved_query', 'target_ids')])(train_samples)

In [5]:
train_dataset = CAsT_Query_ReWriting_Dataset(train_samples, get_query_fn)

In [6]:
train_dataloader = train_dataset.to_dataloader(2)

In [7]:
for batch in train_dataloader:
    print(batch)
    break

{'input_ids': tensor([[    0, 25860,    35,   653,    16, 14599,  1668,   116,     2,     1,
             1,     1,     1,     1],
        [    0,  2264,    16, 14599,  1668,   116, 25860,    35,  1534,    24,
          3951,   868,   116,     2]]), 'input_attention_mask': tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]), 'decoder_input_ids': tensor([[    0,  2264,    16, 14599,  1668,   116,     1],
        [    0,  6209, 14599,  1668,  3951,   868,   116]]), 'decoder_target_ids': tensor([[ 2264,    16, 14599,  1668,   116,     2,     1],
        [ 6209, 14599,  1668,  3951,   868,   116,     2]]), 'target_attention_mask': tensor([[1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1.]])}


In [8]:
batch['input_ids'].shape

torch.Size([2, 14])

In [9]:
model = BART_Query_ReWriter()

Some weights of the model checkpoint at facebook/bart-large were not used when initializing BartForConditionalGeneration: ['encoder.version', 'decoder.version']
- This IS expected if you are initializing BartForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model.training_step(batch,0)

{'loss': tensor(6.2256, grad_fn=<NllLossBackward>)}

In [13]:
my_trainer = Model_Trainer(gpus=[0])
my_trainer.train(model, train_dataloader)

Detected 1 GPUS available, using [0].
Main device is: cuda:0



Keyboard Interrupt!


{'train_loss': [tensor(6.2180, device='cuda:0', grad_fn=<NllLossBackward>),
  tensor(4.8902, device='cuda:0', grad_fn=<NllLossBackward>),
  tensor(6.8974, device='cuda:0', grad_fn=<NllLossBackward>),
  tensor(4.1799, device='cuda:0', grad_fn=<NllLossBackward>),
  tensor(5.9533, device='cuda:0', grad_fn=<NllLossBackward>),
  tensor(5.7294, device='cuda:0', grad_fn=<NllLossBackward>),
  tensor(4.4888, device='cuda:0', grad_fn=<NllLossBackward>),
  tensor(5.6775, device='cuda:0', grad_fn=<NllLossBackward>),
  tensor(5.5300, device='cuda:0', grad_fn=<NllLossBackward>),
  tensor(8.7157, device='cuda:0', grad_fn=<NllLossBackward>),
  tensor(3.4128, device='cuda:0', grad_fn=<NllLossBackward>),
  tensor(5.1156, device='cuda:0', grad_fn=<NllLossBackward>),
  tensor(5.8400, device='cuda:0', grad_fn=<NllLossBackward>),
  tensor(4.7003, device='cuda:0', grad_fn=<NllLossBackward>),
  tensor(4.4362, device='cuda:0', grad_fn=<NllLossBackward>),
  tensor(6.0768, device='cuda:0', grad_fn=<NllLossBackwa

In [43]:
trainer = Trainer(gpus=1, gradient_clip_val=0.5)
trainer.fit(model, train_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name | Type                         | Params
------------------------------------------------------
0 | BART | BartForConditionalGeneration | 406 M 


1